In [1]:
duration :int  = 1.5

In [2]:
import pandas as pd

In [3]:
# import a pickle file
pd_pickle = pd.read_pickle("../pd/2025-03-26_15-13-39_left.pkl")

In [4]:
# add a new column named acc_sum_squared with the sum of the squared values of AccX, AccY, and AccZ
pd_pickle['acc_sum_squared'] = pd_pickle['AccX'] ** 2 + pd_pickle['AccY'] ** 2 + pd_pickle['AccZ'] ** 2

In [5]:
# add 1.5 second to every timestamp in the data
pd_pickle['timestamp'] = pd_pickle['timestamp'] + pd.Timedelta(seconds=1.5)

In [6]:
# delete the rows with timestamp before '2025-03-26 15:05:25.000'
pd_pickle = pd_pickle.iloc[pd_pickle[pd_pickle['timestamp'] == '2025-03-26 15:05:25.000'].index[0]:]

In [7]:
def cut(data, start, end):
    return data[(data['timestamp'] >= start) & (data['timestamp'] <= end)]

In [8]:
# # delete all row after this row with timestamp
# pd_pickle = pd_pickle.iloc[:pd_pickle[pd_pickle['timestamp'] == ''].index[0]]
# # delete all row before this row with timestamp
# pd_pickle = pd_pickle.iloc[pd_pickle[pd_pickle['timestamp'] == ''].index[0]:]

In [9]:
def sure_frequency(data: pd.DataFrame):
    i = 0
    # create an empty list
    frequencies = []
    while True:
        # Calculate the time span in seconds
        time_span = (data['timestamp'].iloc[-(1 + i * 200)] - data['timestamp'].iloc[-(201 + i * 200)]).total_seconds()

        # get the number of records
        number_of_records = 200

        # Calculate the frequency
        frequency = number_of_records / time_span

        # print time span and frequency
        print(f"Time Span: {time_span} seconds")
        print(f"Frequency: {frequency} Hz")
        frequencies.append(frequency)
        i += 1
        if 201 + i * 200 >= len(data):
            break
    # list all different value and corresponding number in frequencies
    from collections import Counter
    print(Counter(frequencies))

In [10]:
print(pd_pickle.shape)
print(pd_pickle.dtypes)
print(pd_pickle.describe())
print(pd_pickle.info)
print(pd_pickle.head())

(99241, 11)
AccX                      float64
AccY                      float64
AccZ                      float64
AsX                       float64
AsY                       float64
AsZ                       float64
AngX                      float64
AngY                      float64
AngZ                      float64
timestamp          datetime64[ns]
acc_sum_squared           float64
dtype: object
               AccX          AccY          AccZ           AsX           AsY  \
count  99241.000000  99241.000000  99241.000000  99241.000000  99241.000000   
mean      -0.022900     -0.013821      0.784907      1.939548      1.648633   
min      -15.946777    -15.000977     -9.305176   -327.026367   -496.520996   
25%       -0.089355      0.001465      0.755371      0.427246     -1.037598   
50%        0.005859      0.008301      0.803711      1.892090      0.732422   
75%        0.047852      0.026855      0.844238      3.967285      2.380371   
max       15.038574      2.434082      7.501953

In [11]:
pd_pickle['timestamp'].diff().describe()
sure_frequency(pd_pickle)

Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 1.0 seconds
Frequency: 200.0 Hz
Time Span: 

In [12]:
correct = ('2025-03-26 15:05:28.000', '2025-03-26 15:08:51.000')

In [13]:
wrong_right = ('2025-03-26 15:09:25.000', '2025-03-26 15:11:19.000')

In [14]:
wrong_left = ('2025-03-26 15:11:43.000', '2025-03-26 15:13:32.000')

In [15]:
import matplotlib.pyplot as plt

def show_column(data, column_name, time_range):
    """
    Plots the specified column data interactively within a given time range.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The column to be plotted.
        time_range (tuple): A tuple containing the start and end timestamps as strings.
    """
    start_time, end_time = time_range
    filtered_data = data[(data['timestamp'] >= start_time) & (data['timestamp'] <= end_time)]

    %matplotlib notebook
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_data['timestamp'], filtered_data[column_name], label=column_name, color='blue')
    plt.title(f'{column_name} over Time')
    plt.xlabel('Timestamp')
    plt.ylabel(column_name)
    plt.legend()
    plt.grid(True)
    plt.show()


In [16]:
def find_max_value_index(df, input_timestamp, column_name, time_window=2):
    """
    Finds the index of the row with the maximum value in the specified column within a given time window before the given timestamp.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the sensor data.
        input_timestamp (str or pd.Timestamp): The reference timestamp.
        column_name (str): The column to search for the maximum value.
        time_window (int): The time window in seconds before the input timestamp to search within. Default is 2 seconds.

    Returns:
        int or None: The index of the row with the maximum value in the specified column within the time range, or None if no such row exists.
    """
    # Ensure the timestamp is in datetime format
    input_timestamp = pd.to_datetime(input_timestamp)

    # Filter data within the specified time window before the input timestamp
    mask = (df['timestamp'] < input_timestamp) & (df['timestamp'] >= input_timestamp - pd.Timedelta(seconds=time_window))
    filtered_df = df[mask]

    # Find the index of the row with the maximum value in the specified column
    if not filtered_df.empty:
        return filtered_df[column_name].idxmax()
    else:
        return None

In [17]:
correct_touch = ['2025-03-26 15:06:17.500', '2025-03-26 15:06:38.800', '2025-03-26 15:06:55.700',
                 '2025-03-26 15:07:13.500', '2025-03-26 15:07:29.500', '2025-03-26 15:07:43.000', '2025-03-26 15:07:58.000',
                 '2025-03-26 15:08:09.800', '2025-03-26 15:08:18.500', '2025-03-26 15:08:29.000', '2025-03-26 15:08:40.300']
# correct_touch = ['2025-03-26 15:06:17.500', '2025-03-26 15:06:38.800', '2025-03-26 15:06:55.700',
#                  '2025-03-26 15:07:13.500', '2025-03-26 15:07:29.500', '2025-03-26 15:07:43.000', '2025-03-26 15:07:58.000',
#                  '2025-03-26 15:08:09.800', '2025-03-26 15:08:18.500', '2025-03-26 15:08:29.000']

In [18]:
# use the function for correct_touch
# save the result in a list named correct_touch_max_accx_indexs
correct_touch_max_accx_indexs : list = []
for timestamp in correct_touch:
    correct_touch_max_accx_indexs.append(find_max_value_index(pd_pickle, timestamp, 'AccX', 2))

In [19]:
correct_touch_max_acc_sum_squared_indexs = []
for timestamp in correct_touch:
    correct_touch_max_acc_sum_squared_indexs.append(find_max_value_index(pd_pickle, timestamp, 'acc_sum_squared', 2))

In [20]:
wrong_right_touch = ['2025-03-26 15:09:28.500', '2025-03-26 15:09:41.000', '2025-03-26 15:09:51.500',
                     '2025-03-26 15:10:02.200', '2025-03-26 15:10:15.000', '2025-03-26 15:10:27.000', '2025-03-26 15:10:37.800', '2025-03-26 15:10:47.500', '2025-03-26 15:10:57.500',
                     '2025-03-26 15:11:11.600']

In [21]:
wrong_left_touch = ['2025-03-26 15:11:54.000',
                    '2025-03-26 15:12:03.000', '2025-03-26 15:12:11.200', '2025-03-26 15:12:18.500', '2025-03-26 15:12:27.500', '2025-03-26 15:12:39.300', '2025-03-26 15:12:49.300',
                    '2025-03-26 15:13:02.500', '2025-03-26 15:13:15.500', '2025-03-26 15:13:26.500']

In [22]:
# create a function that take in a dataframe and a timestamp and return a dataframe with the data from duration seconds before the timestamp to the timestamp
def get_data_before_timestamp(data: pd.DataFrame, time_touch: str, duration: int) -> pd.DataFrame:
    """
    Get the data from a specified duration before a given timestamp.

    Parameters:
        data (pd.DataFrame): The DataFrame containing the sensor data.
        time_touch (str): The reference timestamp.
        duration (int): The duration in seconds before the timestamp.

    Returns:
        pd.DataFrame: The data from the specified duration before the timestamp.
    """
    # Ensure the timestamp is in datetime format
    time_touch = pd.to_datetime(time_touch)

    # Calculate the start time based on the duration
    start_time = time_touch - pd.Timedelta(seconds=duration)

    # Filter the data within the specified time range
    # only return data of the columns 'AccX', 'AccY', 'AccZ', 'AsX', 'AsY', 'AsZ'
    filtered_data = data[(data['timestamp'] > start_time) & (data['timestamp'] <= time_touch)][['AccX', 'AccY', 'AccZ', 'AsX', 'AsY', 'AsZ']]

    return filtered_data

In [23]:
# use the get_data_before_timestamp function for the correct_touch timestamps to get a list of dataframes
correct_touch_dataframes : list = []
for timestamp in correct_touch:
    correct_touch_dataframes.append(get_data_before_timestamp(pd_pickle, timestamp, duration))

In [24]:
import numpy as np

In [25]:
# turn the list of dataframes into a numpy array
correct_touch_numpy: np.array = np.array(correct_touch_dataframes)

In [26]:
correct_touch_numpy.shape

(11, 300, 6)

In [27]:
# at the second dimension, add a numpy array([0, 1, 0, 0, 0, 0]) to the correct_touch_numpy at the beginning
correct_touch_numpy = np.insert(correct_touch_numpy, 0, np.array([[0, 1, 0, 0, 0, 0]]), axis=1)

In [28]:
correct_touch_numpy.shape

(11, 301, 6)

In [29]:
correct_touch_numpy[1, 0]

array([0., 1., 0., 0., 0., 0.])

In [30]:
# use the get_data_before_timestamp function for the wrong_right_touch timestamps to get a list of dataframes
wrong_right_touch_dataframes: list = []
for timestamp in wrong_right_touch:
    wrong_right_touch_dataframes.append(get_data_before_timestamp(pd_pickle, timestamp, duration))

In [31]:
# turn the list of dataframes into a numpy array
wrong_right_touch_numpy: np.array = np.array(wrong_right_touch_dataframes)

In [32]:
wrong_right_touch_numpy.shape

(10, 300, 6)

In [33]:
# at the second dimension, add a numpy array([1, 1, 0, 0, 0, 0]) to the wrong_right_touch_numpy at the beginning
wrong_right_touch_numpy = np.insert(wrong_right_touch_numpy, 0, np.array([[1, 2, 0, 0, 0, 0]]), axis=1)

In [34]:
wrong_right_touch_numpy.shape

(10, 301, 6)

In [35]:
wrong_right_touch_numpy[0, 0]

array([1., 2., 0., 0., 0., 0.])

In [36]:
# use the get_data_before_timestamp function for the wrong_left_touch timestamps to get a list of dataframes
wrong_left_touch_dataframes: list = []
for timestamp in wrong_left_touch:
    wrong_left_touch_dataframes.append(get_data_before_timestamp(pd_pickle, timestamp, duration))

In [37]:
# turn the list of dataframes into a numpy array
wrong_left_touch_numpy: np.array = np.array(wrong_left_touch_dataframes)

In [38]:
wrong_left_touch_numpy.shape

(10, 300, 6)

In [39]:
# at the second dimension, add a numpy array([2, 1, 0, 0, 0, 0]) to the wrong_left_touch_numpy at the beginning
wrong_left_touch_numpy = np.insert(wrong_left_touch_numpy, 0, np.array([[2, 3, 0, 0, 0, 0]]), axis=1)

In [40]:
wrong_left_touch_numpy.shape

(10, 301, 6)

In [41]:
wrong_left_touch_numpy[0, 0]

array([2., 3., 0., 0., 0., 0.])

In [42]:
# combine the correct_touch_numpy, wrong_right_touch_numpy, and wrong_left_touch_numpy
combined_numpy = np.concatenate((correct_touch_numpy, wrong_right_touch_numpy, wrong_left_touch_numpy))

In [43]:
combined_numpy.shape

(31, 301, 6)

In [45]:
# save the combined_numpy_final as a .pkl file using pickle
import pickle
import os
dir = '../classification_forward/data'
# make sure the directory exists
if not os.path.exists(dir):
    os.makedirs(dir)
path = os.path.join(dir, 'combined_numpy_1.pkl')

with open(path, 'wb') as f:
    pickle.dump(combined_numpy, f)

In [220]:
show_column(pd_pickle, 'AccX', ("2025-03-26 15:06:15.500", "2025-03-26 15:06:18.500"))

<IPython.core.display.Javascript object>